# Implementation

## Packages

In [7]:
# data elaboration functions
import pandas as pd
from six.moves import collections_abc
import string
import numpy as np

# datetime functions
import datetime as dt

# file management functions
import os
import sys
import opendatasets as od
import pickle
from pathlib import Path

# plot functions
import matplotlib.pyplot as plt
%matplotlib inline

# data science functions
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, scale
from sklearn.metrics import mean_absolute_error
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# statistical functions
from scipy.stats.mstats import winsorize

# configuration file
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# custom functions
from Code.Profiling.Intermittent.intermittent import Intermittent
from Code.Utils.utils import Utils
from Code.Scoring.kpi import Kpi
from Code.Scoring.forecast import Forecasting
from Code.Scoring.train import Training
from Code.Scoring.train_test import TrainTest
from Code.Scoring.scoring import Scoring
from Code.Regressors.regressors import Regressors
from Code.Plotting.plots import Plots
from Configuration.config import cfg_path

## Setup

In [8]:
# od.download("https://www.kaggle.com/arashnic/building-sites-power-consumption-dataset/download")
root = Path(os.getcwd()).parent
dataset_path = os.path.join(root, cfg_path.data_dir.input_path)

## Load Data

In [9]:
df_final = pd.read_pickle(os.path.join(
    root, cfg_path.data_dir.output_path, 'df_final.pkl'))
df_final.head()

,ICD10Description,DateOfAccident,holidays,wd_mon,wd_tue,wd_wed,wd_thu,wd_fri,wd_sat,wd_sun,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,Amputation_of_finger,1982-10-24,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,Amputation_of_finger,1982-10-25,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Amputation_of_finger,1982-10-26,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Amputation_of_finger,1982-10-27,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Amputation_of_finger,1982-10-28,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Parameter setup

In [10]:
id = 'site_id'
list_unique_id = ['site_id', 'timestamp']
list_temp = ['temp']
y = 'value'
date_var = Utils.find_date(df_final)

find_date, date_col found: ['DateOfAccident']


## Working dataframe

In [11]:
# Defining working dataframe by removing potential null values in y variable
df = df_final.loc[~df_final[y].isnull(),].copy()
print('Id available', list(df[id].unique()))

KeyError: 'value'

# Compute intermittent indicators

In [ ]:
# Winsorizing parameters
highest = 0.05
lowest = 0.05

# Identifying intermittent time series parameters
threshold = 250
perc = 0.01
quant = 0.999

### Choose indicator function

In [ ]:
# Applying indicator function ('compute_indicator_values' or 'enhanced_compute_indicator_values')
func = 'compute_indicator_values'
df_scoring = df.loc[:, [id, y]].groupby(id).apply(lambda x: Intermittent.call_intermittent_function(func,
    np.array(x), threshold, perc, quant, highest, lowest)).reset_index(level=id)

## Profiling intermittent time series

In [1]:
# Excluding the ids for which the indicators are np.NaN
df_scoring_no_nan = df_scoring.dropna()
list_nan = list(set(df_scoring[id]) - set(df_scoring_no_nan[id]))
print('List of nan ids', list_nan)

# Classify intermittent TS for electricity consumption
thres_cv2_constant = 0.06
thres_cv2 = 2
thres_adi = 3
thres_sddi = 6.2
min_time_cons = 2
type = df_scoring_no_nan['type'].unique()[0]

df_profiling = Intermittent.classify_intermittent(df_scoring_no_nan, type, thres_cv2_constant, thres_cv2, thres_adi, thres_sddi, min_time_cons)

# Dictionary
list_of_profiles = ['regular', 'constant_zero', 'constant', 'spikes',
                    'lumpy', 'erratic', 'unforecastable_time', 'unforecastable_quantity']
dict_profiling = {}
for c in list_of_profiles:
    dict_profiling[c] = list(
        df_profiling.loc[df_profiling.profile == c, id].unique())
    print(id, c, dict_profiling[c])


NameError: name 'df_scoring' is not defined

### Plotting profiled series

In [12]:
for profile in list_of_profiles:
    df_to_plot = df.loc[df[id].isin(list(dict_profiling[profile])), ].copy()
    count = 1
    for i in list(df_to_plot[id].unique()):
        print('Plotting id:', i, 'as', count, 'of',
              len(list(df_to_plot[id].unique())))
        plot = Plots.sliding_line_plot(df_to_plot, y, id, i, chart_title="")
        folder_name = os.path.join(root, cfg_path.data_dir.plot_path, profile)
        Utils.create_folder_tree(folder_name)
        plot.write_html(os.path.join(folder_name, id + '_' + str(i) + '_profile_' + profile + ".html"))
        count = count + 1


Plotting id: 2 as 1 of 35
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 2
Successfully created the directory c:\Users\mabellani\OneDrive - Microsoft\Documents\REPOSITORY\dstoolkit-forecasting\Data/Plots\regular
Plotting id: 6 as 2 of 35
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 6
Creation of the directory failed or already present c:\Users\mabellani\OneDrive - Microsoft\Documents\REPOSITORY\dstoolkit-forecasting\Data/Plots\regular
Plotting id: 8 as 3 of 35
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 8
Creation of the directory failed or already present c:\Users\mabellani\OneDrive - Microsoft\Documents\REPOSITORY\dstoolkit-forecasting\Data/Plots\regular
Plotting id: 9 as 4 of 35
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 9
Creation of the directory failed or already present c:\Users\mabellani\OneDrive - Microsoft\Documents\REPOSITOR

# Saving

In [10]:
# create a binary pickle file 
f = open(os.path.join(root, cfg_path.data_dir.output_path, 'dict_profiling.pkl'),"wb")
# write the python object (dict) to pickle file
pickle.dump(dict_profiling,f)
# close file
f.close()